# **Import Modules**

In [58]:
import os
import numpy as np
import cv2
import tensorflow as tf
from gtts import gTTS
import pygame
from keras.api.applications import ResNet50V2
from keras.api.models import Sequential
from keras.api.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.api.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.api.optimizers import SGD
from sklearn.model_selection import train_test_split

# **Define Path, Image Size, and Batch**

In [4]:
dataset_path = '/Users/geoffrey/Downloads/Banknotes Dataset'

IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# **Load Dataset**

In [5]:
# Dataset load function
def load_images_from_folder(parent_folder):
    images = []
    labels = []
    class_names = set()
    for country_folder in os.listdir(parent_folder):
        country_path = os.path.join(parent_folder, country_folder)

        if os.path.isdir(country_path):
            for banknote_folder in os.listdir(country_path):
                banknote_path = os.path.join(country_path, banknote_folder)

                if os.path.isdir(banknote_path):
                    label = f"{country_folder}_{banknote_folder}"
                    class_names.add(label)

                    for filename in os.listdir(banknote_path):
                        img_path = os.path.join(banknote_path, filename)
                        img = cv2.imread(img_path)

                        if img is not None:
                            img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
                            images.append(img)
                            labels.append(label)

    return np.array(images), np.array(labels), sorted(class_names)

# Load dataset
images, labels, class_names = load_images_from_folder(dataset_path)

# **Train the Model**

In [6]:
# Convert labels to numeric values
label_to_index = {name: index for index, name in enumerate(class_names)}
labels = np.array([label_to_index[label] for label in labels])

# Normalize images
images = images / 255.0

#Load the pre-trained ResNet50V2 model without the top layer
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

#Freeze the base model
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(images, labels, epochs=20, batch_size=BATCH_SIZE, validation_split=0.2)

Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.0257 - loss: 4.4956 - val_accuracy: 0.0000e+00 - val_loss: 5.0831
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.0625 - loss: 4.0986 - val_accuracy: 0.0000e+00 - val_loss: 5.7932
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.1254 - loss: 3.6794 - val_accuracy: 0.0000e+00 - val_loss: 6.8298
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.1995 - loss: 3.2008 - val_accuracy: 0.0000e+00 - val_loss: 7.3775
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.2994 - loss: 2.7110 - val_accuracy: 0.0000e+00 - val_loss: 8.8555
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.3593 - loss: 2.3553 - val_accuracy: 0.0000e+00 - val_loss: 9.6364
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.4594 - loss: 2.0288 - val_accuracy: 0.0000e+00 - val_loss: 10.2335
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5572 - loss: 1.6139 - val

# **Get the Exchange Rate**

In [7]:
def get_exchange_rates():
    exchange_rates = {
        'USD' : {
            'Rupiah' : 16234.00,
            'Peso' : 58.50,
            'SingaporeDollar' : 1.35,
            'Ringgit' : 4.68,
            'Dong' : 25457.50,
            'Baht' : 36.70,
            'Kip' : 21495.00,
            'NewTaiwanDollar' : 32.37,
            'Kyat' : 2100.08,
            'BruneiDollar' : 1.35,
            'Riel' : 4093.00,
        },

        'EUR' : {
            'Rupiah' : 17584.88,
            'Peso' : 63.37,
            'SingaporeDollar' : 1.46,
            'Ringgit' : 5.08,
            'Dong' : 27576.84,
            'Baht' : 39.76,
            'Kip' : 23284.46,
            'NewTaiwanDollar' : 35.07,
            'Kyat' : 2275.13,
            'BruneiDollar' : 1.46,
            'Riel' : 4434.56,
        }
    }

    return exchange_rates

# **Detect the Currency**

In [76]:
def detect_currency(frame, model, class_names):
    img = cv2.resize(frame, (IMG_HEIGHT, IMG_WIDTH))
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    currency_label = class_names[class_index]

    return currency_label

# Initialize pygame for playing audio
pygame.init()

# Function to capture an image from webcam and detect currency
def capture_image_and_detect_currency():
    # Capture video from webcam
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            break
        
        # Display the resulting frame
        cv2.imshow('Capture Image', frame)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the capture
    cap.release()
    cv2.destroyAllWindows()
    
    # Detect currency
    currency_label = detect_currency(frame, model, class_names)
    
    # Parse the detected currency
    country_name, banknote_value = currency_label.split('_')
    country = country_name.split(' ')
    banknote_value_num, banknote_name = banknote_value.split(' ')

    # Get current exchange rates
    exchange_rates = get_exchange_rates()
    usd_rate = exchange_rates['USD'][banknote_name]
    eur_rate = exchange_rates['EUR'][banknote_name]
    
    # Calculate equivalent values
    value_in_usd = float(banknote_value_num) / usd_rate
    value_in_eur = float(banknote_value_num) / eur_rate
    
    # Prepare the speech output
    speech_output = (f"The currency you are holding is {country} with the amount of {banknote_value_num} {banknote_name}. "
                     f"Here are some details about its exchange value: "
                     f"1 USD equals to {usd_rate} {banknote_name}, while 1 EURO equals to {eur_rate} {banknote_name}. So,"
                     f"{banknote_value_num} {banknote_name} is equal to {value_in_usd:.2f} USD and {value_in_eur:.2f} EURO.")
    
    # Convert text to speech using gTTS
    tts = gTTS(text=speech_output, lang='en')
    tts.save("speech.mp3")
    
    # Play the audio using pygame
    pygame.mixer.music.load("speech.mp3")
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

# Capture an image and detect currency
capture_image_and_detect_currency()

KeyboardInterrupt: 